In [1]:
!pip install -q transformers torchvision pandas pillow tqdm

In [1]:
# ==========================================
# 1. SETUP
# ==========================================
!pip install -q transformers torchvision pandas pillow tqdm

import sys
import torch
import torch.nn as nn
import os
from torch.utils.data import DataLoader
from torchvision import models
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Connect to your shared modular file
sys.path.append('/content/drive/My Drive/AdMIRe_Project/')
import admire_dataset
import importlib
importlib.reload(admire_dataset) # Reload to be safe
from admire_dataset import AdMIReReader

# ==========================================
# 2. LOAD DATASET
# ==========================================
EXTRACT_PATH = "/content/admire_data"

print("📂 Initializing Fusion Reader...")
try:
    # mode='fusion' tells the reader to return PIXEL TENSORS
    # The reader will automatically find 'subtask_a_train.tsv' in the subfolder
    train_dataset = AdMIReReader(data_root_path=EXTRACT_PATH, split="Train", mode="fusion")
    print(f"✅ Fusion Dataset Loaded: {len(train_dataset)} items")
except Exception as e:
    print(f"❌ Error: {e}")
    raise

📂 Initializing Fusion Reader...
🕵️ Scanning /content/admire_data for Train data...
✅ Loaded: /content/admire_data/train/subtask_a_train.tsv
✅ Fusion Dataset Loaded: 70 items


In [2]:
# ==========================================
# 3. DEFINE MODEL (ResNet50 + DistilBERT)
# ==========================================
class IdiomFusionNet(nn.Module):
    def __init__(self):
        super(IdiomFusionNet, self).__init__()

        # --- TOWER 1: VISION (ResNet50) ---
        # Load pre-trained ResNet and remove the classification head
        resnet = models.resnet50(pretrained=True)
        self.vision_encoder = nn.Sequential(*list(resnet.children())[:-1]) # Output: 2048 dims
        self.vision_proj = nn.Linear(2048, 256) # Project to 256 dims

        # --- TOWER 2: TEXT (DistilBERT) ---
        self.text_model = AutoModel.from_pretrained("distilbert-base-uncased")
        self.text_proj = nn.Linear(768, 256) # Project to 256 dims

    def forward(self, images, input_ids, attention_mask):
        """
        images: [Batch, 5, 3, 224, 224]
        input_ids: Text tokens
        """
        batch_size = images.shape[0]

        # A. ENCODE TEXT
        text_out = self.text_model(input_ids=input_ids, attention_mask=attention_mask)
        text_emb = text_out.last_hidden_state[:, 0, :] # [CLS] token
        text_emb = self.text_proj(text_emb) # [Batch, 256]

        # B. ENCODE IMAGES
        # Flatten batch to process 5 images at once
        images_flat = images.view(-1, 3, 224, 224)

        img_feats = self.vision_encoder(images_flat).squeeze() # [Batch*5, 2048]
        img_emb = self.vision_proj(img_feats) # [Batch*5, 256]

        # Reshape back to [Batch, 5, 256]
        img_emb = img_emb.view(batch_size, 5, 256)

        # C. FUSION (Dot Product)
        text_emb_expanded = text_emb.unsqueeze(1) # [Batch, 1, 256]
        scores = torch.sum(text_emb_expanded * img_emb, dim=2) # [Batch, 5]

        return scores

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from tqdm import tqdm
import pandas as pd

# ==========================================
# 4. TRAINING CONFIGURATION
# ==========================================
# We initialize the tokenizer here for the collate function
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def fusion_collate(batch):
    images = torch.stack([item['pixel_values'] for item in batch])
    labels = torch.stack([item['labels'] for item in batch])
    texts = [item['raw_text'] for item in batch]

    # Tokenize text
    toks = tokenizer(texts, padding=True, truncation=True, max_length=64, return_tensors="pt")

    # Return texts too so we can use them in the final report
    return images, toks['input_ids'], toks['attention_mask'], labels, texts

# DataLoader (Batch Size 16 is safe for T4 GPU)
BATCH_SIZE = 16
loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=fusion_collate)

# Setup Device & Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = IdiomFusionNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# ==========================================
# 5. TRAINING LOOP
# ==========================================
EPOCHS = 5
print(f"\n🚀 Starting Full Training on {device}...")
print(f"⚡ Configuration: {len(train_dataset)} items | Batch Size {BATCH_SIZE} | {EPOCHS} Epochs")

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    progress = tqdm(loader, desc=f"Epoch {epoch+1}/{EPOCHS}")

    for imgs, input_ids, mask, labels, _ in progress:
        imgs, input_ids, mask, labels = imgs.to(device), input_ids.to(device), mask.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward Pass
        logits = model(imgs, input_ids, mask)

        # Loss Calculation
        loss = criterion(logits, labels)

        # Backward Pass
        loss.backward()
        optimizer.step()

        # Metrics
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        total_loss += loss.item()

        # Live Progress Bar Update
        progress.set_postfix({"Loss": loss.item(), "Acc": correct/total})

    # Epoch Summary
    avg_loss = total_loss / len(loader)
    final_acc = correct / total
    print(f"✅ Epoch {epoch+1} Done. Avg Loss: {avg_loss:.4f} | Accuracy: {final_acc:.4f}")

# Save the Model
save_path = "/content/drive/My Drive/AdMIRe_Project/fusion_model_full.pth"
torch.save(model.state_dict(), save_path)
print(f"💾 Trained Model Saved to: {save_path}")

# ==========================================
# 6. GENERATE READABLE REPORT
# ==========================================
print("\n📝 Generating Full Report (Evaluation Pass)...")
model.eval()
results = []

# Create a non-shuffled loader for the final report
report_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=fusion_collate)

with torch.no_grad():
    for imgs, input_ids, mask, labels, raw_texts in tqdm(report_loader, desc="Evaluating"):
        imgs, input_ids, mask, labels = imgs.to(device), input_ids.to(device), mask.to(device), labels.to(device)

        # Get Scores
        logits = model(imgs, input_ids, mask)
        probs = F.softmax(logits, dim=1) # Convert to % confidence
        preds = torch.argmax(logits, dim=1)

        for i in range(len(labels)):
            truth = labels[i].item()
            prediction = preds[i].item()
            confidence = probs[i][prediction].item() * 100

            results.append({
                "Idiom_Sentence": raw_texts[i],
                "True_Image_No": truth + 1,       # Convert index 0 -> Image 1
                "Predicted_Image_No": prediction + 1,
                "Result": "CORRECT" if truth == prediction else "WRONG",
                "Confidence": f"{confidence:.1f}%"
            })

# Save to Drive
report_file = "/content/drive/My Drive/AdMIRe_Project/fusion_full_report.tsv"
df = pd.DataFrame(results)

print("\n" + "="*60)
print(f"📊 FINAL REPORT SUMMARY")
print(f"Total Items: {len(df)}")
print(f"Overall Accuracy: {(df['Result']=='CORRECT').mean()*100:.2f}%")
print("-" * 60)
print("Preview (First 10 Rows):")
print(df[['True_Image_No', 'Predicted_Image_No', 'Result', 'Confidence']].head(10))
print("="*60)

df.to_csv(report_file, sep='\t', index=False)
print(f"💾 Full readable report saved to: {report_file}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R


🚀 Starting Full Training on cuda...
⚡ Configuration: 70 items | Batch Size 16 | 5 Epochs


Epoch 1/5: 100%|██████████| 5/5 [00:17<00:00,  3.56s/it, Loss=1.46, Acc=0.229]


✅ Epoch 1 Done. Avg Loss: 1.8498 | Accuracy: 0.2286


Epoch 2/5: 100%|██████████| 5/5 [00:14<00:00,  2.95s/it, Loss=0.0454, Acc=0.8]


...

✅ Epoch 4 Done. Avg Loss: 0.0708 | Accuracy: 0.9571


Epoch 5/5: 100%|██████████| 5/5 [00:15<00:00,  3.02s/it, Loss=7.95e-5, Acc=0.957]


✅ Epoch 5 Done. Avg Loss: 0.0691 | Accuracy: 0.9571
💾 Trained Model Saved to: /content/drive/My Drive/AdMIRe_Project/fusion_model_full.pth

📝 Generating Full Report (Evaluation Pass)...


Evaluating: 100%|██████████| 5/5 [00:20<00:00,  4.10s/it]


📊 FINAL REPORT SUMMARY
Total Items: 70
Overall Accuracy: 95.71%
------------------------------------------------------------
Preview (First 10 Rows):
   True_Image_No  Predicted_Image_No   Result Confidence
0              1                   1  CORRECT     100.0%
1              2                   2  CORRECT     100.0%
2              3                   3  CORRECT     100.0%
3              4                   4  CORRECT     100.0%
4              4                   4  CORRECT     100.0%
5              2                   2  CORRECT     100.0%
6              5                   5  CORRECT     100.0%
7              1                   1  CORRECT     100.0%
8              2                   1    WRONG      20.0%
9              4                   4  CORRECT     100.0%
💾 Full readable report saved to: /content/drive/My Drive/AdMIRe_Project/fusion_full_report.tsv
